In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import yfinance as yf
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Just_normal/'

Mounted at /content/drive


In [ ]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [ ]:
filename = 'CCV_data.csv'
CCV_data = pd.read_csv(path + filename)

In [ ]:
set_index(CCV_data)

In [ ]:
data = pd.DataFrame(columns=['garch', 'return'], index=CCV_data.index)

In [ ]:
data['garch'] = CCV_data["('GARCH', 'normal', 0)"].values

In [ ]:
data['return'] = CCV_data["log_returns"].values

In [ ]:
data['CCV'] = CCV_data["CCV"].values

In [ ]:
data

,garch,return,CCV
Date,,,
2000-02-02,0.800313,-0.027564,0.866853
2000-02-03,0.655577,0.893688,0.528602
2000-02-04,0.491617,-0.052641,0.528672
2000-02-07,0.408321,-0.025746,0.528836
2000-02-08,0.339040,0.976294,0.374766
...,...,...,...
2024-08-26,0.093281,-0.275848,0.366574
2024-08-27,-0.162765,0.111656,0.366496
2024-08-28,-0.126100,-0.507014,0.370549


# Split train, validation and test data

In [ ]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  return train_data, val_data, test_data

In [ ]:
def sliding_window_transform(X, y, timesteps):
  X_windows = []
  y_windows = []
  for i in range(len(X) - timesteps + 1):
    X_window = X[i:i + timesteps]
    y_window = y[i + timesteps - 1]
    X_windows.append(X_window)
    y_windows.append(y_window)

  return np.array(X_windows), np.array(y_windows)

In [ ]:
# Define the rolling window function
def rolling_window(df, in_sample_window_size, out_of_sample_size):
  X, y = [], []
  for i in range(in_sample_window_size, len(df) - out_of_sample_size):
    X.append(df.iloc[i - in_sample_window_size:i, :-1].values)  # All features except the target column
    y.append(df.iloc[i:i + out_of_sample_size, -1].values)  # Target column

  return np.array(X), np.array(y)

# Model

In [ ]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

## CCV

In [ ]:
daily_train, daily_val, daily_test = split_data(data, 'CCV')

In [ ]:
in_sam_win_sz = 25
out_sam_win_sz = 5

In [ ]:
CCV_X_train, CCV_y_train = rolling_window(daily_train, in_sam_win_sz, out_sam_win_sz)
CCV_X_val, CCV_y_val = rolling_window(daily_val, in_sam_win_sz, out_sam_win_sz)
CCV_X_test, CCV_y_test = rolling_window(daily_test, in_sam_win_sz, out_sam_win_sz)

In [ ]:
# CCV_X_train = CCV_X_train.reshape(CCV_X_train.shape[0], win_sz, -1)
# CCV_X_val = CCV_X_val.reshape(CCV_X_val.shape[0], win_sz, -1)
# CCV_X_test = CCV_X_test.reshape(CCV_X_test.shape[0], win_sz, -1)

### LSTM

In [ ]:
lstm_garch_model = Sequential()
lstm_garch_model.add(LSTM(48, input_shape=(CCV_X_train.shape[1], CCV_X_train.shape[2]), return_sequences=True))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(LSTM(16))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(Dense(5))

lstm_garch_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
lstm_garch_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 25, 48)              │           9,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,037 (54.83 KB)

 Trainable params: 14,037 (54.83 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
lstm_history = lstm_garch_model.fit(CCV_X_train, CCV_y_train, epochs=80, batch_size=32, validation_data=(CCV_X_val, CCV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - loss: 0.2449 - val_loss: 0.0845
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0530 - val_loss: 0.0516
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - loss: 0.0489 - val_loss: 0.0532
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 0.0377 - val_loss: 0.0429
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0439 - val_loss: 0.1600
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - loss: 0.0673 - val_loss: 0.0555
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 0.0383 - val_loss: 0.0493
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0378 - val_loss: 0.0509
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0372 - val_loss: 0.0421
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - loss: 0.0327 - val_loss: 0.0411
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0342 - val_loss: 0.0358
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Predictions
CCV_test_predictions = lstm_garch_model.predict(CCV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step


In [ ]:
mse = mean_squared_error(CCV_y_test, CCV_test_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(CCV_y_test, CCV_test_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(CCV_y_test, CCV_test_predictions)
print(f'MAPE: {mape}%')

MSE: 0.027442135415692376
RMSE: 0.16565667935731532
MAE: 0.1216061293930425
MAPE: 189.19681365013375%


### GRU

In [ ]:
gru_model = Sequential()
gru_model.add(GRU(48, input_shape=(CCV_X_train.shape[1], CCV_X_train.shape[2]), return_sequences=True))
gru_model.add(Dropout(0.1))
gru_model.add(GRU(16))
gru_model.add(Dropout(0.1))
gru_model.add(Dense(5))

gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
gru_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 25, 48)              │           7,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,741 (41.96 KB)

 Trainable params: 10,741 (41.96 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
gru_history = gru_model.fit(CCV_X_train, CCV_y_train, epochs=80, batch_size=32, validation_data=(CCV_X_val, CCV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - loss: 0.1664 - val_loss: 0.0462
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.0562 - val_loss: 0.0689
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - loss: 0.0435 - val_loss: 0.0459
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.0552 - val_loss: 0.0553
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 0.0410 - val_loss: 0.0460
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - loss: 0.0421 - val_loss: 0.0404
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - loss: 0.0332 - val_loss: 0.0353
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - loss: 0.0339 - val_loss: 0.0347
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.0343 - val_loss: 0.0498
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.0333 - val_loss: 0.0743
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - loss: 0.0387 - val_loss: 0.0565
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 

In [34]:
# Predictions
CCV_gru_predictions = gru_model.predict(CCV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [35]:
mse = mean_squared_error(CCV_y_test, CCV_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(CCV_y_test, CCV_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(CCV_y_test, CCV_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 0.019576692754296767
RMSE: 0.13991673507588992
MAE: 0.09603041011214451
MAPE: 87.73907794294944%


### LSTM_GRU

In [36]:
lstm_gru_model = Sequential()
lstm_gru_model.add(LSTM(48, input_shape=(CCV_X_train.shape[1], CCV_X_train.shape[2]), return_sequences=True))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(GRU(16))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(Dense(5))

lstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [37]:
lstm_gru_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 25, 48)              │           9,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,045 (50.96 KB)

 Trainable params: 13,045 (50.96 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
lstm_gru_history = lstm_garch_model.fit(CCV_X_train, CCV_y_train, epochs=80, batch_size=32, validation_data=(CCV_X_val, CCV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 0.0175 - val_loss: 0.0477
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0141 - val_loss: 0.0552
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - loss: 0.0200 - val_loss: 0.0475
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0171 - val_loss: 0.0520
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.0153 - val_loss: 0.0516
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0176 - val_loss: 0.0537
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0146 - val_loss: 0.0518
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: 0.0143 - val_loss: 0.0485
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0165 - val_loss: 0.0487
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 0.0130 - val_loss: 0.0471
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0146 - val_loss: 0.0661
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 

In [39]:
# Predictions
CCV_lstm_gru_predictions = lstm_garch_model.predict(CCV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [40]:
mse = mean_squared_error(CCV_y_test, CCV_lstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(CCV_y_test, CCV_lstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(CCV_y_test, CCV_lstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 0.024285378918238225
RMSE: 0.15583766848306677
MAE: 0.11725453689796525
MAPE: 200.85580433369051%
